# BOT DE TELEGRAM<br>

In [ ]:
token = TOKEN

import os
import telebot
import data_preparation
import feature_extractor
import pandas as pd
from preprocess import preprocess
from url_parser import parse_url
from classificator import predict

PROJECT_ROOT_PATH = "."
MODELS_PATH = PROJECT_ROOT_PATH + os.sep + "modelos"

#logger = telebot.logger
#telebot.logger.setLevel(logging.DEBUG) # Outputs debug messages to console.

bot = telebot.TeleBot(token)

#message_handler detecta en Telegram el comando asociado. Ej: /start
#La variable message almacena el mensaje que acompaña a cada comando
#Ej: /detect https://www.google.es 
@bot.message_handler(commands=['start'])
def send_welcome(message):
    ini = "Hola, envíame una URL y comprobaré si es maliciosa o no! \nUtiliza el comando /detect <url> para comenzar. \nSi necesitas ayuda ejecuta el comando /help ."
    #La función reply_to envía un mensaje al usuario de Telegram
    bot.reply_to(message, ini)
    
@bot.message_handler(commands=['help'])
def send_help(message):
    help_message = "Bot capaz de detectar URLs maliciosas. Los comandos son: \n\b- /detect <url> : comprueba si una URL es maliciosa o no \n\b- /stop : detiene el bot \n\b- /help : muestra esta ayuda"
    bot.reply_to(message, help_message)
    
@bot.message_handler(commands=['detect'])
def handle_url(message):
    url = str(message.text)
    urls = url.split()
    first_url = urls[1]
    if not data_preparation.contains_scheme(first_url):
            first_url = 'http://' + first_url
    if not data_preparation.url_is_valid(first_url):
        error_msg = "URL no válida. ¡Por favor, introduzca una URL válida!."
        bot.reply_to(message, error_msg)
    else:
        checking_url_msg = "Comprobando URL. Espere unos segundos...!"
        bot.reply_to(message, checking_url_msg)
        first_url = data_preparation.url_to_lowercase(first_url)
        #First technique
        most_abused_tlds, suspect_extensions = feature_extractor.initialize_resources()
        features = feature_extractor.extract_features(first_url, most_abused_tlds, suspect_extensions)
        preprocess_features_1 = preprocess(features)
        columns = ['hostname_depth','domain_length', 'hostname_length', 'n_special', 'www_prefix', 'vowel_consonant_ratio', 'digit_character_ratio', 'bad_tld', 'malicious_extension']
        df_features_1 = pd.DataFrame([preprocess_features_1], columns=columns)
        #Second technique
        df_parts_url_2 = parse_url(first_url)
        prediction = predict(df_features_1, df_parts_url_2)
        if prediction == 0:
            bot.reply_to(message, "URL no maliciosa")
        else:
            bot.reply_to(message, "¡URL maliciosa!")    

bot.polling(none_stop=True)